In [108]:
#!pip install msgpack
#!pip install funcy
#!pip install pyspark==2.3.0


In [8]:
# import os
# os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[2] pyspark-shell"
# os.environ["JAVA_HOME"] = "C:\jdk1.8.0_171"
#os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-10.0.1"


#USATO PER RISOLVEREE PROBLEMA JAVA GATEWAY ..... (ho anche installato java e inserito manualmente java home tra le variabili ambiente)

In [9]:
# print (os.getenv("JAVA_HOME"))

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
from pyspark.sql.functions import monotonically_increasing_id
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
import pyspark
import string
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.mllib.util import MLUtils
from pyspark.sql.types import *
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
nltk.download("stopwords")
import funcy as fp
import pyLDAvis

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\r.parisi.cons\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
spark = SparkSession.builder.appName('NLP_topicModel').getOrCreate()


In [3]:
rawdata = spark.read.json("ARTICOLITV2.json", multiLine=True)
rawdata = rawdata.fillna({'testo_txt_it': ''})
rawdata = rawdata.withColumn("uid", monotonically_increasing_id())
rawdata = rawdata.withColumn("year_month", rawdata.published_dt.substr(1,7))
rawdata.show(3)


+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+---+----------+
|          _version_|                  id|language|language_s|        published_dt|        testo_txt_it|               url_s|uid|year_month|
+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+---+----------+
|1600612814188183552|2cfcaaac52cfbb66d...|        |   italian|2018-05-15T14:08:00Z|| 0
Dal 1° al 10 ...|http://omgili.com...|  0|   2018-05|
|1600612814388461568|0cf575e54190bff2e...|        |   italian|2018-05-15T14:08:00Z|15 maggio 2018 16...|http://omgili.com...|  1|   2018-05|
|1600612814394753024|b8dbc4e7826b41575...|        |   italian|2018-05-15T14:08:00Z|Leonardo Nappi 13...|http://omgili.com...|  2|   2018-05|
+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+---+----------+
only showing 

In [13]:
#def getrows(df, rownums=None):
#     return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: x[0])
#getrows(rawdata.select("testo_txt_it"), [2]).collect()
#rawdata.select("testo_txt_it").rdd.zipWithIndex().filter(lambda x: x[1] in [2]).map(lambda x: x[0])
#rawdata.select("testo_txt_it").show(3, False)

In [14]:
#sc = SparkContext.getOrCreate()
#sc._conf.getAll()

In [15]:
#rawdata.select("testo_txt_it").show(3, False)#

# Puliamo i dati:
- Applichiamo la tokenization
- Rimuoviamo le stopwords. A questo insieme 

In [4]:
def myStemmer (record, array=False):
    stemmer = SnowballStemmer("italian")
    if array:
        text_out = [stemmer.stem(word) for word in record] 
    else:    
        text = record [2]#9
        text_out = [stemmer.stem(word) for word in text]
    return(text_out)

    

#def myCleaner (record):
#    text = record [10]
#    text_out = [re.sub("[^a-zA-Z]","", word.strip().lower()) for word in text]
#    text_out = [word for word in text if (word != "" and word != " ")]
#    return(text_out)

def removePunctuation(column):
    return trim(lower(regexp_replace(column, '[^A-Za-z]', ' '))).alias('sentence')

In [17]:
# print("Contains digits",contains_digits)
# print("more_then_3_charachters",more_then_3_charachters)


In [5]:

prova = rawdata.select(removePunctuation(col("testo_txt_it")))
#Tokenize the text in the text column
tokenizer = Tokenizer(inputCol="sentence", outputCol="tokenized")#testo_txt_it
wordsDataFrame = tokenizer.transform(prova)#rawdata

remover = StopWordsRemover(inputCol="tokenized", outputCol="tokenizedNew", stopWords = [" ", ""])
wordsDataFrame = remover.transform(wordsDataFrame)

udf_myStemmer = udf(myStemmer, (ArrayType(StringType()))) # if the function returns an int
wordsDataFrame = wordsDataFrame.withColumn("stemmed", udf_myStemmer(struct([wordsDataFrame[x] for x in wordsDataFrame.columns])))

# udf_myFunction = udf(myCleaner, (ArrayType(StringType()))) # if the function returns an int
# wordsDataFrame = wordsDataFrame.withColumn("cleaned", udf_myFunction(struct([wordsDataFrame[x] for x in wordsDataFrame.columns])))

# #remove 20 most occuring documents, documents with non numeric characters, and documents with <= 3 characters
cv_tmp = CountVectorizer(inputCol="stemmed", outputCol="tmp_vectors")
cv_tmp_model = cv_tmp.fit(wordsDataFrame)

#wordsDataFrame.show(5)

stopWords = list(set(stopwords.words('english')))+list(set(stopwords.words('italian')))

top100 = list(cv_tmp_model.vocabulary[0:100])
stopWordsCustom = [" ","", "dal", "al","davan","avev","qualc", "qualcuno", "qualcosa", "avevano", "davanti", "aveva","e","avere", "fare","la","li", "lo", "gli", "essere", "solo", "per", "cosa", "ieri","disponibile", "anno", "detto", "quando","fatto", "sotto", "alcuna", "quali"]
#Add additional stopwords in th, is list

more_then_3_charachters = [word for word in cv_tmp_model.vocabulary if len(word) <= 3 ]
stopWords = list(set(stopwords.words('english')))+list(set(stopwords.words('italian')))
#Combine all the stopwords
stpw = StopWordsRemover.loadDefaultStopWords("italian") + stopWords+ stopWordsCustom +top100 + more_then_3_charachters 
stem_stopw = myStemmer(stpw, True) #stemming the stopwords

#Remove stopwords from the tokenized list
removerNew = StopWordsRemover(inputCol="stemmed", outputCol="final", stopWords =   stem_stopw+stpw) 
clean_text= removerNew.transform(wordsDataFrame)#dropping the stemmed stopwords from the stemmed word

In [8]:
wordsDataFrame.select("Stemmed").show(5)

+--------------------+
|             Stemmed|
+--------------------+
|[dal, al, giugn, ...|
|[magg, condivisio...|
|[leonard, napp, m...|
|[twitter, exop, o...|
|[mir, debutt, av,...|
+--------------------+
only showing top 5 rows



In [20]:
# reviews = data.map(lambda x : x['Review Text']).filter(lambda x: x is not None)
# StopWords = stopwords.words("english")
# tokens = reviews                                                   \
#     .map( lambda document: document.strip().lower())               \
#     .map( lambda document: re.split(" ", document))          \
#     .map( lambda word: [x for x in word if x.isalpha()])           \
#     .map( lambda word: [x for x in word if len(x) > 3] )           \
#     .map( lambda word: [x for x in word if x not in StopWords])    \
#     .zipWithIndex()

clean_text.select("final").show(5, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
# def stem(in_vec):
#     out_vec = []
#     for t in in_vec:
#         t_stem = stemmer.stem(t)
#         if len(t_stem) > 2:
#             out_vec.append(t_stem)       
#     return out_vec

# # Create user defined function for stemming with return type Array<String>
# from pyspark.sql.types import *
# stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))
# # Create new df with vectors containing the stemmed tokens 
# wordsDataFrame = (
#     wordsDataFrame
#         .withColumn("vector_stemmed", stemmer_udf("filtered"))
#         .select("vector_stemmed")
#   )

# # Rename df and column for clarity
# production_df = wordsDataFrame.select(col("vector_stemmed").alias("unigrams"))

# # Display
# production_df.printSchema()
# production_df.show()


In [6]:
train, test = clean_text.select("final").randomSplit([0.75, 0.25])
train.show(5)
test.show(5)

+--------------------+
|               final|
+--------------------+
|                  []|
|[accant, sardegn,...|
|[acciaier, tropp,...|
|[acqua, econom, b...|
|[adozion, distanz...|
+--------------------+
only showing top 5 rows

+--------------------+
|               final|
+--------------------+
|[agricol, friulad...|
|[aics, casert, vi...|
|[alasd, macleod, ...|
|[alvis, biff, pre...|
|[ancor, luig, fav...|
+--------------------+
only showing top 5 rows



In [ ]:
#train = train.na.drop(how="all")
#train = train.drop(train.take(1))
#train.show()

In [7]:
#clean_text.select("final").show(3)
#cv = CountVectorizer(inputCol="final", outputCol="rawFeatures", vocabSize = 4000, minDF = 5)
#cvmodel = cv.fit(clean_text)
#featurizedData = cvmodel.transform(clean_text)
#featurizedData.select("final","rawFeatures").show(5)
cv_train = CountVectorizer(inputCol="final", outputCol="rawFeatures", vocabSize = 1000, minDF = 5)
cvmodel = cv_train.fit(train)
cvDatasetTrain = cvmodel.transform(train)
cvDatasetTrain.select("final","rawFeatures").show(5)
cvDatasetTest = cvmodel.transform(test)

+--------------------+--------------------+
|               final|         rawFeatures|
+--------------------+--------------------+
|                  []|        (1000,[],[])|
|[accant, sardegn,...|(1000,[12,13,15,1...|
|[acciaier, tropp,...|(1000,[1,7,16,20,...|
|[acqua, econom, b...|(1000,[0,1,3,5,8,...|
|[adozion, distanz...|(1000,[1,2,3,5,7,...|
+--------------------+--------------------+
only showing top 5 rows



In [8]:
# vocab = cvmodel.vocabulary
# vocab_broadcast = spark.sparkContext.broadcast(vocab)
idf = IDF(inputCol="rawFeatures", outputCol="features")
#idfModel = idf.fit(featurizedData)
#rescaledData = idfModel.transform(featurizedData)
idfModel = idf.fit(cvDatasetTrain)
rescaledDataTrain = idfModel.transform(cvDatasetTrain)
rescaledDataTest = idfModel.transform(cvDatasetTest)


In [18]:
#rescaledData.select("features").show(2, False)

In [9]:
# Num_Argomenti = [10, 20, 50, 100, 150]
# loglik=[]
# logperp=[]
# n_iter = {Num_Argomenti[0]: 20, Num_Argomenti[1]: 40, Num_Argomenti[2]: 100, Num_Argomenti[3]:180, Num_Argomenti[4]: 260}
# for i in Num_Argomenti:
#     print(i)
#   http://localhost:8888/notebooks/Desktop/Materiale%20didattico/LDA/MIO_ULTIMO.ipynb#  lda = LDA(k=i, seed=123, optimizer="online", featuresCol="features", optimizeDocConcentration= True, maxIter= n_iter[i])
#     ldamodel = lda.fit(rescaledData)
#     loglik.append(ldamodel.logLikelihood(rescaledData))
#     logperp.append(ldamodel.logPerplexity(rescaledData))
k = 10
lda = LDA(k=k, seed=123, optimizer="online",featuresCol="features", maxIter = 10)# optimizeDocConcentration= True, subsamplingRate = 0.2, learningDecay = 0.5, 
#ldanew = LDA(k=15, seed=123, optimizer="online",featuresCol="features", subsamplingRate = 0.2,maxIter = 20)# optimizeDocConcentration= True, subsamplingRate = 0.2, learningDecay = 0.5, 
#ldamodelNEW = ldanew.fit(rescaledDataTrain)

ldamodel = lda.fit(rescaledDataTrain)
#loglik.append(ldamodel.logLikelihood(rescaledData))
#logperp.append(ldamodel.logPerplexity(rescaledData))
ldatopics = ldamodel.describeTopics()
ldatopics.show(k)


+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[142, 49, 426, 18...|[0.01437681460424...|
|    1|[69, 90, 100, 501...|[0.00739121668728...|
|    2|[183, 177, 185, 3...|[0.04449776966721...|
|    3|[213, 208, 236, 3...|[0.02372797475481...|
|    4|[548, 2, 245, 62,...|[0.01315361616498...|
|    5|[994, 891, 837, 6...|[0.01879038131770...|
|    6|[82, 329, 379, 10...|[0.01359071323028...|
|    7|[999, 202, 191, 5...|[0.01571794009143...|
|    8|[93, 358, 285, 37...|[0.01642307686275...|
|    9|[366, 262, 396, 3...|[0.01933120692758...|
+-----+--------------------+--------------------+



In [ ]:
prediction = ldamodel.transform(rescaledDataTest)
predictionTrain = ldamodel.transform(rescaledDataTrain)

In [171]:
def extract_data (ldamodel, countVectorizer_transf, countVectorizer_fit, df = train, results_pred = predictionTrain, column = "rawFeatures"):
    vocab = countVectorizer_fit.vocabulary
    rows_contents = countVectorizer_transf.select(column).take(df.count())
    doc_lengths=[]
    term_frequencies_list = []
    for i in range(len(rows_contents)):
        sparse_vector = rows_contents[i].asDict()[column] #è uno sparse vector se printato visualizza solo elem nonzero e
                                                    #rappresenta l'indice parola e la sua freq all'interno del documento
        doc_lengths.append(sparse_vector.numNonzeros()) #numero elementi non zero all'interno dell'array (senza le parentesi da 
                                                        # l'indice degli elem nonzero)
        term_frequencies_list.append(sparse_vector.toArray()) #toArray() mostra il vettore per intero, compresi gli zeri
    term_frequency = list(np.sum(term_frequencies_list, axis = 0)) #andiamo a sommare i vettori dei vari documenti, per ottenere le freq nel corpus
    to_drop = [i for i in range(len(doc_lengths)) if doc_lengths[i] == 0]       
    doc_lengths = [doc_lengths[i] for i in range(len(doc_lengths)) if i not in to_drop]   
    word_dists_topic = np.asmatrix(ldamodel.topicsMatrix().toArray())
    #qui abbiamo per ogni parola la distribuzione nei vari topic (1000, 10), noi vogliamo l'opposto (10, 1000)
    #quindi andiamo a trasporre la matrice appena creata e a trasformarla in una lista di liste (formato desiderato)
    topic_term_dists = word_dists_topic.transpose().tolist()
    pred_top_dists_pd = results_pred.select("topicDistribution").toPandas()
    doc_topic_dists = [list(pred_top_dists_pd.iloc[i][0]) for i in range(results_pred.count())]
    doc_topic_dists = [doc_topic_dists[i] for i in range(len(doc_topic_dists)) if i not in to_drop]       
#     results_visual = {"vocab": vocab, 'doc_lengths': doc_lengths , 'term_frequency': term_frequency, 'doc_topic_dists': doc_topic_dists, 'topic_term_dists': topic_term_dists}
    return(vocab, doc_lengths, term_frequency, topic_term_dists, doc_topic_dists)

In [169]:
vocab, doc_lengths, term_frequency, topic_term_dists, doc_topic_dists = extract_data (ldamodel, cvDatasetTrain, cvmodel, train, predictionTrain, "rawFeatures")


In [173]:
prep_visual = pyLDAvis.prepare(topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency )

# topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency, R=30, lambda_step=0.01, mds=<function js_PCoA>, n_jobs=-1, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, sort_topics=True)

In [175]:
pyLDAvis.display(prep_visual)

In [ ]:
pyLDAvis.show(prep_visual, local = False)

In [106]:
#Parametri da aggiungere
#subsamplingRate = compreso tra 0 e 1
#learningDecay = learning rate deve essere compreso tra 0.5 e 1 per raggiungere la convergenza
print("Train-set results")
print("LogLikelihood", ldamodel.logLikelihood(rescaledDataTrain), "LogPerplexity", ldamodel.logPerplexity(rescaledDataTrain))

Train-set results
LogLikelihood -1128702.8324757658 LogPerplexity 6.812628133263962


In [107]:
# import numpy as np
# plt.plot(Num_Argomenti, loglik)
# print(logperp)
# print(loglik)
print("Test-set results")
print("LogLikelihood", ldamodel.logLikelihood(rescaledDataTest), "LogPerplexity", ldamodel.logPerplexity(rescaledDataTest))

Test-set results
LogLikelihood -394452.6543559207 LogPerplexity 7.122864899285956


In [26]:
# vocab = cvmodel.vocabulary
# # vocab_broadcast = spark.sparkContext.broadcast(vocab)
# ldatopics = ldamodel.describeTopics()
# ldatopics.show(150)
# def map_termID_to_Word(termIndices):
#     words = []
#     for termID in termIndices:
#         words.append(vocab_broadcast.value[termID])
    
#     return words

# udf_map_termID_to_Word = udf(map_termID_to_Word , ArrayType(StringType()))
# ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))
# ldaResults = ldamodel.transform(rescaledData)
# ldatopics_mapped.select(ldatopics_mapped.topic, ldatopics_mapped.topic_desc).show(150,False)

+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[2027, 1180, 2157...|[0.01313742846803...|
|    1|[154, 104, 663, 4...|[0.00752504958309...|
|    2|[444, 198, 2116, ...|[0.00952783037172...|
|    3|[310, 152, 326, 8...|[0.00498978966448...|
|    4|[232, 1630, 970, ...|[0.01838347049476...|
|    5|[465, 1426, 36, 1...|[0.00592561698570...|
|    6|[238, 648, 222, 2...|[0.01145734486457...|
|    7|[1028, 473, 264, ...|[0.00792293385259...|
|    8|[554, 1252, 1753,...|[0.01119429585335...|
|    9|[1348, 446, 1043,...|[0.01271373133341...|
|   10|[293, 198, 296, 2...|[0.02211430248833...|
|   11|[545, 455, 2267, ...|[0.00911897278426...|
|   12|[34, 60, 26, 35, ...|[0.00304525944350...|
|   13|[1577, 1811, 654,...|[0.01662764158489...|
|   14|[397, 819, 245, 6...|[0.01641437607184...|
+-----+--------------------+--------------------+

+-----+------------------------------------------

In [61]:
predictionTrain.select("topicDistribution").show(5)

+--------------------+
|   topicDistribution|
+--------------------+
|[0.0,0.0,0.0,0.0,...|
|[3.92633679060754...|
|[5.35829935615589...|
|[1.63941981519168...|
|[0.60520190248780...|
+--------------------+
only showing top 5 rows

